<a href="https://colab.research.google.com/github/tokaew01/MusicDB-TOT-2/blob/main/Hands-on-1/SENT_v3_Hands_on_1_Splitter_and_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ติดตั้ง LlamaIndex และ dependencies
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q

# Import modules
import os
import urllib.request
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import MarkdownNodeParser, SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ฟังก์ชันดาวน์โหลด corpus
def download_corpus():
    import urllib.request
    import os
    os.makedirs('./corpus_input', exist_ok=True)
    urls = [
        ("https://storage.googleapis.com/llm-course/md/1.md", "./corpus_input/1.md")
    ]
    for url, path in urls:
        if not os.path.exists(path):
            print(f"Downloading {url} to {path}")
            try:
                urllib.request.urlretrieve(url, path)
            except Exception as e:
                print(f"Failed to download {url}: {e}")

# ดาวน์โหลด corpus
print("กำลังดาวน์โหลดไฟล์...")
download_corpus()

# โหลดเอกสาร Markdown จากไดเรกทอรี
reader = SimpleDirectoryReader(
    input_dir="./corpus_input",
    recursive=True,
    required_exts=[".md", ".markdown"]
)
documents = reader.load_data()
print(f"โหลดเอกสาร {len(documents)} ไฟล์สำเร็จ")

##############################################
# ส่วนที่ 1: การเปรียบเทียบระหว่าง Parser และ Splitter
##############################################

print("\n=== การเปรียบเทียบระหว่าง MarkdownNodeParser กับ SentenceSplitter ===\n")

# สร้าง parsers และ splitters
md_parser = MarkdownNodeParser()
sentence_splitter = SentenceSplitter(chunk_size=800, chunk_overlap=50)

# เปรียบเทียบผลลัพธ์สำหรับแต่ละไฟล์
for doc in documents:
    filename = doc.metadata.get('file_name', 'Unknown')
    print(f"\n>> ไฟล์: {filename}")

    # ทดสอบ MarkdownNodeParser
    parser_nodes = md_parser.get_nodes_from_documents([doc])

    # ทดสอบ SentenceSplitter
    splitter_nodes = sentence_splitter.get_nodes_from_documents([doc])

    # เปรียบเทียบจำนวน nodes
    print(f"MarkdownNodeParser: {len(parser_nodes)} nodes")
    print(f"SentenceSplitter: {len(splitter_nodes)} nodes")

    # แสดงตัวอย่าง nodes จาก Parser
    print(f"\nตัวอย่าง nodes จาก MarkdownNodeParser (3 ตัวอย่างแรก):")
    for i, node in enumerate(parser_nodes[:1]):
        print(f"\n  Node {i+1}:")
        print(f"  Header Path: {node.metadata.get('header_path', 'No header path')}")
        print(f"  Text: {node.text}")

    # แสดงตัวอย่าง nodes จาก Splitter
    print(f"\nตัวอย่าง nodes จาก SentenceSplitter (3 ตัวอย่างแรก):")
    for i, node in enumerate(splitter_nodes[:3]):
        print(f"\n  Node {i+1}:")
        print(f"  Text: {node.text}")

    # แสดงตัวอย่างเฉพาะสำหรับเนื้อหาที่มีหัวข้อชัดเจน (สำหรับไฟล์ 1.md)
    if filename == "1.md":
        print("\n=== ตัวอย่างเฉพาะ: การเปรียบเทียบเนื้อหาเกี่ยวกับ 'สาเหตุของโรคหัดเยอรมัน' ===")

        # หา nodes ที่เกี่ยวกับ "สาเหตุของโรคหัดเยอรมัน" จาก MarkdownNodeParser โดยค้นหาจาก text
        cause_parser_nodes = [node for node in parser_nodes if "สาเหตุของโรคหัดเยอรมัน" in node.text]
        if cause_parser_nodes:
            print("\nMarkdownNodeParser: เนื้อหาสาเหตุของโรคหัดเยอรมันอยู่ใน node เดียว")
            node = cause_parser_nodes[0]
            print(f"  Header Path: {node.metadata.get('header_path', 'No header path')}")
            print(f"  Text: {node.text}")

        # หา nodes ที่เกี่ยวกับ "สาเหตุของโรคหัดเยอรมัน" จาก SentenceSplitter (อาจกระจายอยู่หลาย nodes)
        cause_splitter_nodes = [node for node in splitter_nodes if "สาเหตุของโรคหัดเยอรมัน" in node.text]
        print(f"\nSentenceSplitter: พบ {len(cause_splitter_nodes)} nodes ที่มีคำว่า 'สาเหตุของโรคหัดเยอรมัน'")
        for i, node in enumerate(cause_splitter_nodes[:2]):  # แสดงเพียง 2 nodes
            print(f"\n  Node {i+1}:")
            print(f"  Text: {node.text}")

# สรุปข้อแตกต่างระหว่าง Parser และ Splitter
print("""
=== สรุปข้อแตกต่างระหว่าง MarkdownNodeParser และ SentenceSplitter ===

1. การแบ่ง nodes:
   - MarkdownNodeParser: แบ่งตามโครงสร้างของ Markdown (ตามหัวข้อ #, ##, ###) รักษาความสัมพันธ์ของเนื้อหาไว้
   - SentenceSplitter: แบ่งตามขนาดที่กำหนด (chunk_size) โดยพยายามรักษาขอบเขตของประโยค ไม่สนใจโครงสร้าง Markdown

2. Metadata:
   - MarkdownNodeParser: เก็บข้อมูลเกี่ยวกับพาธของหัวข้อ (header_path) ซึ่งแสดงลำดับชั้นของหัวข้อ
   - SentenceSplitter: ไม่มีข้อมูลเกี่ยวกับโครงสร้างของเอกสาร ทำให้อาจสูญเสียบริบทของหัวข้อ

3. จำนวน Nodes:
   - MarkdownNodeParser: น้อยกว่า เพราะแบ่งตามหัวข้อเท่านั้น
   - SentenceSplitter: มากกว่า เพราะแบ่งตามขนาดที่กำหนด

4. ลักษณะการทำงาน:
   - MarkdownNodeParser: เน้นการรักษาโครงสร้างและบริบทของหัวข้อ
   - SentenceSplitter: เน้นการควบคุมขนาดของ chunks ให้มีความสม่ำเสมอ

5. การกำหนดค่า:
   - MarkdownNodeParser: ไม่ต้องกำหนดค่า chunk_size หรือ chunk_overlap
   - SentenceSplitter: ต้องกำหนดค่า chunk_size และ chunk_overlap เพื่อควบคุมขนาด node
""")

##############################################
# ส่วนที่ 2: ขั้นตอนการ Ingest จนถึงการสร้าง Indexes
##############################################

print("\n=== ขั้นตอนการ Ingest จนถึงการสร้าง Indexes ===")

# 1. สร้าง Indexes จากแต่ละ parser
print("\n1. การสร้าง Indexes จากแต่ละ parser")

# สร้าง nodes จาก MarkdownNodeParser
md_nodes = md_parser.get_nodes_from_documents(documents)
print(f"  MarkdownNodeParser: สร้าง {len(md_nodes)} nodes")

# ตั้งค่า embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# สร้าง index สำหรับ MarkdownNodeParser
md_index = VectorStoreIndex(nodes=md_nodes)

# สร้าง nodes จาก SentenceSplitter
sentence_nodes = sentence_splitter.get_nodes_from_documents(documents)
print(f"  SentenceSplitter: สร้าง {len(sentence_nodes)} nodes")

# สร้าง index สำหรับ SentenceSplitter
sentence_index = VectorStoreIndex(nodes=sentence_nodes)

กำลังดาวน์โหลดไฟล์...
โหลดเอกสาร 4 ไฟล์สำเร็จ

=== การเปรียบเทียบระหว่าง MarkdownNodeParser กับ SentenceSplitter ===


>> ไฟล์: 1.md
MarkdownNodeParser: 17 nodes
SentenceSplitter: 31 nodes

ตัวอย่าง nodes จาก MarkdownNodeParser (3 ตัวอย่างแรก):

  Node 1:
  Header Path: /
  Text: # หัดเยอรมัน อาการ สาเหตุ และการรักษาโรคหัดเยอรมัน 10 วิธี !!

โดย เมดไทย, เมื่อ 27 กรกฎาคม 2020 (เวลา 18:30 น.)

ตัวอย่าง nodes จาก SentenceSplitter (3 ตัวอย่างแรก):

  Node 1:
  Text: # หัดเยอรมัน อาการ สาเหตุ และการรักษาโรคหัดเยอรมัน 10 วิธี !!

โดย เมดไทย, เมื่อ 27 กรกฎาคม 2020 (เวลา 18:30 น.)

## โรคหัดเยอรมัน

หัดเยอรมัน, เหือด หรือ หัดสามวัน (German measles/เจอร์มันมีเซิลส์, Rubella/รูเบลลา หรือ Three-day measles/ทรีเดย์มีเซิลส์) เป็นโรคไข้ออกผื่นที่เกิดจากการติดเชื้อไวรัสหัดเยอรมัน ผู้ป่วยจะมีอาการไข้และออกผื่นคล้ายโรคหัด แต่จะมีความรุนแรงและโรคแทรกซ้อนน้อยกว่าหัด โรคนี้ไม่ใช่โรคร้ายแรง ถ้าเป็นกับเด็กหรือผู้ใหญ่ทั่วไป มักจะหายได้เองโดยไม่มีโรคแทรกซ้อนที่รุนแรง แต่ถ้าเกิดในหญิงตั้งครรภ์ในช่วงไตรมาสแรกขอ